<a href="https://colab.research.google.com/github/AkshayAneja222/AkshayAneja222/blob/main/LIHQ_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure a GPU runtime is enabled.

If you plan to actually run each example one after the other, you will either need to revise audio folder contents (leave only one empty audio folder, 'Folder1') between each example or just disconnect and reconnect to reset everything.

# **1) SET UP**

In [1]:
#Git repos

#LIHQ
!git clone https://github.com/johnGettings/LIHQ

%cd LIHQ
#Tortoise
!git clone https://github.com/neonbjb/tortoise-tts.git
#FOMM
!git clone https://github.com/AliaksandrSiarohin/first-order-model
#Wav2Lip
!git clone https://github.com/Rudrabha/Wav2Lip.git
#GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
#QVI
!git clone https://github.com/xuxy09/QVI
#MODNet
!git clone https://github.com/ZHKKKe/MODNet


###LIHQ
!pip install -r requirements.txt

###Tortoise
%mv '/content/LIHQ/tortoise-tts' '/content/LIHQ/tortoise_tts'
%cd /content/LIHQ/tortoise_tts
%cd tortoise-tts
!pip3 install --upgrade setuptools
!pip install transformers==4.31.0 # Installing the specific transformers version
!pip install tokenizers>=0.14,<0.19 #installing required version
!pip3 install -r requirements.txt
!pip3 install setuptools_rust

!pip3 install einops==0.5.0
!pip3 install rotary_embedding_torch==0.1.5
!pip3 install unidecode==1.3.5
!python3 setup.py install

###FOMM
%mv '/content/LIHQ/first-order-model' '/content/LIHQ/first_order_model'
%cd /content/LIHQ/first_order_model
#getting model weights
!gdown --id 1DbjXD2nS3jlyCWoJu2HGcLZZjhLC9a2J

###Wav2Lip
%cd /content/LIHQ/Wav2Lip
#Downloading model weights
!gdown --id 1eAtM-Ck5RMyMMZoQuoQfYZRU5vDDwBpK -O './checkpoints/wav2lip_gan.pth'
!gdown --id 1eAtM-Ck5RMyMMZoQuoQfYZRU5vDDwBpK -O './checkpoints/wav2lip.pth'
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "./face_detection/detection/sfd/s3fd.pth"

#GFPGAN
%cd /content/LIHQ/GFPGAN
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

###QVI
%cd /content/LIHQ/QVI
!wget https://www.dropbox.com/s/5auirpk1tijxo1v/model.pt?dl=0 -O './model.pt'
!wget https://www.dropbox.com/s/afsqhlzu0rarpmf/pwc-checkpoint.pt?dl=0 -O './utils/pwc-checkpoint.pt'
%rm './demo.py'
!gdown --id 1xl7ziToar2ZDNpToOOKc8fugw1xeQ6fp -O './demo.py'

###MODNET
%cd /content/LIHQ/MODNet
!gdown --id 1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz -O pretrained/modnet_photographic_portrait_matting.ckpt

%cd /content/LIHQ

fatal: destination path 'LIHQ' already exists and is not an empty directory.
/content/LIHQ
fatal: destination path 'tortoise-tts' already exists and is not an empty directory.
fatal: destination path 'first-order-model' already exists and is not an empty directory.
fatal: destination path 'Wav2Lip' already exists and is not an empty directory.
fatal: destination path 'GFPGAN' already exists and is not an empty directory.
fatal: destination path 'QVI' already exists and is not an empty directory.
fatal: destination path 'MODNet' already exists and is not an empty directory.
ERROR: Invalid requirement: 'tokenizers=0.21': Expected end or semicolon (after name and no valid version specifier)
    tokenizers=0.21
              ^ (from line 9 of requirements.txt)
Hint: = is not a valid operator. Did you mean == ?
mv: cannot move '/content/LIHQ/tortoise-tts' to '/content/LIHQ/tortoise_tts/tortoise-tts': Directory not empty
/content/LIHQ/tortoise_tts
/content/LIHQ/tortoise_tts/tortoise-tts
DEPR

# **Example 1: Simple Speaker with StyleGAN2 and TorToiSe (Recommended Use)**

** bold textGenerating audio from TorToiSe** (Or you can always put your own audio in the audio folders)

In [6]:
!pip install tokenizers==0.14 # Installs tokenizers within the specified version range

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/addict-2.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/tb_nightly-2.19.0a20250123-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/yapf-0.43.0-py3.11.egg is deprecated. pip 24.3 will enforce this b

In [1]:
# Run this once. Takes about a minute
%cd /content/LIHQ/tortoise_tts
!pip install transformers==4.37.2 # Installing the specific transformers version
!pip install tokenizers==0.18.0 #installing required version
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

from LIHQ.procedures.tortoise_scripts import tortoise_run, tortoise_combo_run

tts = TextToSpeech()

/content/LIHQ/tortoise_tts
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/addict-2.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/tb_nightly-2.19.0a20250123-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/yapf-0.43.0-py3.11.egg is deprecated. p

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Generating a random voice to speak the sentence below. When you generate one you like, save it using the cell below
text = "Look how easy it is to create an artificial speaker named katrina."
voice = 'random'
preset = "high_quality" #Options: {"ultra_fast", "fast", "standard", "high_quality"}.

gen = tortoise_run(tts, text, voice, preset)
IPython.display.Audio(gen.squeeze(0).cpu(), rate = 24000)

rlg_auto.pth:   0%|          | 0.00/25.2M [00:00<?, ?B/s]

/content/LIHQ/tortoise_tts/tortoise/api.py:305: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.rlg_auto.load_state_dict(torch.load(get_model_path('rlg_auto.pth', self.mo

rlg_diffuser.pth:   0%|          | 0.00/101M [00:00<?, ?B/s]

/content/LIHQ/tortoise_tts/tortoise/api.py:307: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.rlg_diffusion.load_state_dict(torch.load(get_model_path('rlg_diffuser.pth'

Generating autoregressive samples..


100%|██████████| 16/16 [08:43<00:00, 32.75s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:14<00:00,  1.10it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [00:37<00:00, 10.57it/s]


In [10]:
# Saving audio from above. Just change the foldername & filename as needed
import os

# Create the directory if it doesn't exist
os.makedirs('/content/LIHQ/input/audio/Folder1/', exist_ok=True)

# Saving audio from above. Just change the foldername & filename as needed
torchaudio.save('/content/LIHQ/input/audio/Folder1/2.wav', gen.squeeze(0).cpu(), 24000)

**Silence Padding**

In [11]:
# Adding half a second of silence before audio, because FOMM works better when first frame is a closed mouth.
# Also helps with abrupt start/ stop if you want to add silence after as well.
from pydub import AudioSegment

silence_duration = 500  # Set duration in milliseconds. 1000 is 1 second
save_path = '/content/LIHQ/input/audio/Folder1/1.wav'

silent_segment = AudioSegment.silent(duration=silence_duration)
silent_segment.export(save_path, format="wav")

<_io.BufferedRandom name='/content/LIHQ/input/audio/Folder1/1.wav'>

**Running main script**

In [14]:
%cd /content/LIHQ
from runLIHQ import run
import traceback

try:
    run(face='/content/LIHQ/input/face/examples/2372448.png')
except SystemExit as e:
    print(f"SystemExit with code: {e}")
    traceback.print_exc()  # This line prints the full stack trace
except Exception as e:
    print(f"Another exception occurred: {e}")
    traceback.print_exc()  # This line prints the full stack trace for other exceptions as well

/content/LIHQ
Initializing
Moviepy - Building video ./first_order_model/input-ref-vid/Folder1/Folder1.mp4.
Moviepy - Writing video ./first_order_model/input-ref-vid/Folder1/Folder1.mp4



Moviepy - Done !
Moviepy - video ready ./first_order_model/input-ref-vid/Folder1/Folder1.mp4
Moviepy - Building video ./first_order_model/input-ref-vid/Sister/Sister.mp4.
Moviepy - Writing video ./first_order_model/input-ref-vid/Sister/Sister.mp4



Moviepy - Done !
Moviepy - video ready ./first_order_model/input-ref-vid/Sister/Sister.mp4
Moviepy - Building video ./first_order_model/input-ref-vid/Sister2/Sister2.mp4.
Moviepy - Writing video ./first_order_model/input-ref-vid/Sister2/Sister2.mp4



Moviepy - Done !
Moviepy - video ready ./first_order_model/input-ref-vid/Sister2/Sister2.mp4
Running First Order Motion Model


  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

FOMM Success!
Running Wav2Lip
Wav2Lip could not generate at least one of your videos.
Possibly bad audio, unrecognizable mouth, bad file paths, out of memory.
Run below command in a separate cell to get full traceback.
###########################################################
###########################################################
import os
adir = 'Folder1' # The audio folder that failed. See Wav2Lip output folder to see whats missing.

vid_path = f'{os.getcwd()}/output/FOMM/Round1/{adir}.mp4'
aud_path = f'{os.getcwd()}/input/audio/{adir}/{adir}.wav'
%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0


SystemExit with code: 


Traceback (most recent call last):
  File "<ipython-input-14-594b93b6c7ff>", line 6, in <cell line: 0>
    run(face='/content/LIHQ/input/face/examples/2372448.png')
  File "/content/LIHQ/runLIHQ.py", line 88, in run
    sys.exit()
SystemExit


In [26]:
import os
adir = 'Folder1' # The audio folder that failed. See Wav2Lip output folder to see whats missing.\n\n'
vid_path = f'/content/LIHQ/first_order_model/input-ref-vid/Folder1/Folder1.mp4'
import os
adir = 'Folder1' # The audio folder that failed. See Wav2Lip output folder to see whats missing.\n\n'
vid_path = f'/content/LIHQ/first_order_model/input-ref-vid/Folder1/Folder1.mp4'
aud_path = f'/content/LIHQ/input/audio/Folder1/Folder1.wav'

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print(vid_path)
print(aud_path)

%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0
print(aud_path)

%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0

%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0
print(aud_path)

%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0

%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0
print(aud_path)

%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0

%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0
print(aud_path)

%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0

%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0
print(aud_path)

%cd /content/LIHQ/Wav2Lip
!python inference.py --checkpoint_path ./checkpoints/wav2lip.pth --face {vid_path} --audio {aud_path} --outfile /content/test.mp4  --pads 0 20 0 0

/content/LIHQ/first_order_model/input-ref-vid/Folder1/Folder1.mp4
/content/LIHQ/input/audio/Folder1/Folder1.wav
/content/LIHQ/Wav2Lip
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 95
/content/LIHQ/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 304)
Length of mel chunks: 92
  0% 0/1 [00:00<?, ?it/s]/content/LIHQ/Wav2Lip/face_detection/detection/sfd/sfd_detector.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `we

# **Example 2: Multiple Speakers, Replacing Background**

**Generating audio from TorToiSe**

In [3]:
# Renaming default audio folder 'Folder1' and creating a second folder, one for each speaker.
# Result will be two output videos
%mv '/content/LIHQ/input/audio/Folder1' '/content/LIHQ/input/audio/Sister'
%mkdir '/content/LIHQ/input/audio/Sister2'

In [1]:
# Run this once. Takes about a minute
%cd /content/LIHQ/tortoise_tts
!pip install transformers==4.37.2 # Installing the specific transformers version
!pip install tokenizers==0.18.0 #installing required version
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

from LIHQ.procedures.tortoise_scripts import tortoise_run, tortoise_combo_run

tts = TextToSpeech()

/content/LIHQ/tortoise_tts
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/addict-2.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/tb_nightly-2.19.0a20250123-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/yapf-0.43.0-py3.11.egg is deprecated. p

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.o

...

/content/LIHQ/tortoise_tts/tortoise/api.py:227: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.diffusion.load_state_dict(torch.load(get_model_path('diffusion_decoder.pth

clvp2.pth:   0%|          | 0.00/976M [00:00<?, ?B/s]

/content/LIHQ/tortoise_tts/tortoise/api.py:233: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.clvp.load_state_dict(torch.load(get_model_path('clvp2.pth', models_dir)))


vocoder.pth:   0%|          | 0.00/391M [00:00<?, ?B/s]

/content/LIHQ/tortoise_tts/tortoise/api.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.vocoder.load_state_dict(torch.load(get_model_path('vocoder.pth', models_di

In [2]:
# Audio for Speaker 1
# Choose one of the provided voices.
text = "Hey, have you seen my sister anywhere?"
voice = 'train_daws'
preset = "standard" #Options: {"ultra_fast", "fast", "standard", "high_quality"}.

gen = tortoise_run(tts, text, voice, preset)
IPython.display.Audio(gen.squeeze(0).cpu(), rate = 24000)

/content/LIHQ/tortoise_tts/tortoise/models/arch_util.py:314: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mel_norms = torch.load(self.mel_norm_file)


Generating autoregressive samples..


100%|██████████| 16/16 [02:59<00:00, 11.19s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:14<00:00,  1.13it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 200/200 [00:14<00:00, 13.78it/s]


In [3]:
# Saving audio from above to folder 'Sister'
torchaudio.save('/content/LIHQ/input/audio/Sister/2.wav', gen.squeeze(0).cpu(), 24000)

In [4]:
# Audio for Speaker 2
text = "Sister? I don't have a sister. Leave me alone."
voice = 'angie'
preset = "standard" #Options: {"ultra_fast", "fast", "standard", "high_quality"}.

gen = tortoise_run(tts, text, voice, preset)
IPython.display.Audio(gen.squeeze(0).cpu(), rate = 24000)

Generating autoregressive samples..


100%|██████████| 16/16 [06:03<00:00, 22.72s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:14<00:00,  1.13it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 200/200 [00:17<00:00, 11.15it/s]


In [5]:
# Saving audio from above to folder 'Sister2'
torchaudio.save('/content/LIHQ/input/audio/Sister2/2.wav', gen.squeeze(0).cpu(), 24000)

In [13]:
# Install pydub using the python environment directly

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install pydub

# Rest of your code
from pydub import AudioSegment

silence_duration = 500  # Set duration in milliseconds. 1000 is 1 second
save_path = '/content/LIHQ/input/audio/Sister/1.wav'

silent_segment = AudioSegment.silent(duration=silence_duration)
silent_segment.export(save_path, format="wav")

save_path = '/content/LIHQ/input/audio/Sister2/1.wav'

silent_segment = AudioSegment.silent(duration=silence_duration)
silent_segment.export(save_path, format="wav")
silence_duration = 500  # Set duration in milliseconds. 1000 is 1 second
save_path = '/content/LIHQ/input/audio/Sister/1.wav'

silent_segment = AudioSegment.silent(duration=silence_duration)
silent_segment.export(save_path, format="wav")

save_path = '/content/LIHQ/input/audio/Sister2/1.wav'

silent_segment = AudioSegment.silent(duration=silence_duration)
silent_segment.export(save_path, format="wav")

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/addict-2.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/tb_nightly-2.19.0a20250123-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/progressbar-2.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/yapf-0.43.0-py3.11.egg is deprecated. pip 24.3 will enforce this b

<_io.BufferedRandom name='/content/LIHQ/input/audio/Sister2/1.wav'>

**Change Background of 'Sister2' (Image Matting)**

In [14]:
#This will create an image mask for every face in the 'LIHQ/input/face/MODNet/input' folder
#So let's copy the image we want for our sister2 to that folder
%cp '/content/LIHQ/input/face/examples/2237618.png' '/content/LIHQ/input/face/MODNet/input/2237618.png'

%cd '/content/LIHQ/MODNet'
%rm '/content/LIHQ/input/face/MODNet/input/.gitkeep'
%rm -d '/content/LIHQ/input/face/MODNet/input/.ipynb_checkpoints'
!python -m demo.image_matting.colab.inference \
        --input-path /content/LIHQ/input/face/MODNet/input --output-path /content/LIHQ/input/face/MODNet/masks \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt

/content/LIHQ/MODNet
rm: cannot remove '/content/LIHQ/input/face/MODNet/input/.ipynb_checkpoints': No such file or directory
/content/LIHQ/MODNet/demo/image_matting/colab/inference.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on G

In [15]:
from LIHQ.procedures.matting_scripts import image_matting

# Combining mask, source, and background
# We will use a backgorund I included in the repo
# Check the .input/face/MODNet/output folder for result
background_image = '/content/LIHQ/input/background/cute.jpg'
face_image = '/content/LIHQ/input/face/MODNet/input/2237618.png'
##### Only chnage these two variables above

mask_folder = '/content/LIHQ/input/face/MODNet/masks'
output_folder = '/content/LIHQ/input/face/MODNet/output'
image_matting(background_image, face_image, mask_folder, output_folder)

  if event.key is 'enter':



**Running Main script**

In [17]:
%cd /content/LIHQ
from runLIHQ import run

# face variable can be an array. ['path/to/face/sister', 'path/to/face/sister2']
# The order should be the same order that you see the audio folders listed in the files window to the left.
face = ['/content/LIHQ/input/face/examples/2237618.png', '/content/LIHQ/input/face/MODNet/output/matte_2237618.png']

# If you do not want the two clips to have the same exact head movement, change the ref_vid_offset for each.
# LIHQ will crop reference video starting at this point (in seconds)
ref_vid_offset = [0, 220]


run(face=face, ref_vid_offset=ref_vid_offset)

/content/LIHQ
Initializing
Moviepy - Building video ./first_order_model/input-ref-vid/Sister/Sister.mp4.
Moviepy - Writing video ./first_order_model/input-ref-vid/Sister/Sister.mp4



Moviepy - Done !
Moviepy - video ready ./first_order_model/input-ref-vid/Sister/Sister.mp4
Moviepy - Building video ./first_order_model/input-ref-vid/Sister2/Sister2.mp4.
Moviepy - Writing video ./first_order_model/input-ref-vid/Sister2/Sister2.mp4



Moviepy - Done !
Moviepy - video ready ./first_order_model/input-ref-vid/Sister2/Sister2.mp4
Running First Order Motion Model


  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

  checkpoint = torch.load(checkpoint_path)



  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

FOMM Success!
Running Wav2Lip
Wav2Lip Complete
Beginning restoration and upscaling
Finished Sister
Finished Sister2
Completed Restoration Round 1
Round 1 Complete!
Round1 Elapsed Time:
94.81467485427856
Beginning Round 2
First Order Motion Model


FileNotFoundError: No such file: '/content/LIHQ/output/frames2Vid/Round1/Sister.mp4'

# **Example 3: Switch out TorToiSe for VITS, use own speaker image, utilize frame interpolation**

**Generating Audio with VITS**

In [ ]:
#Setup. Run Once
%cd '/content'
!git clone https://github.com/jaywalnut310/vits.git
%cd '/content/vits'

!pip install phonemizer==2.2.1
!pip install Unidecode==1.1.1
!pip install pydub

#Setup
%cd /content/vits/monotonic_align
!python setup.py build_ext --inplace

!sudo apt-get install espeak -y

%cd /content/vits
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import sys
import os
import cv2
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from pydub import AudioSegment

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write

!gdown --id 1ZvqoymRV0L15N6StSSw7h9E0ApivPB3u -O '/content/vits/pretrained_vctk.pth'

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

%cd /content/LIHQ/vits
hps = utils.get_hparams_from_file("/content/vits/configs/vctk_base.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint('/content/vits/pretrained_vctk.pth', net_g, None)

In [ ]:
# Write out the text you want spoken
# Choose you speaker id (sid variable below)
  # Can be a value between 0 and 109. Male and female speakers, with UK accents.
stn_tst = get_text("Biscuits and coriander and chips and all that. Cheerio.", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([93]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.95, noise_scale_w=0.8, length_scale=1.2)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

In [ ]:
#save audio to folder
# We save as 2.wav because we want silence padding before the speech.
audio = ipd.Audio(audio, rate=hps.data.sampling_rate)
audio = AudioSegment(audio.data, frame_rate=22050, sample_width=2, channels=1)
savePath = '/content/LIHQ/input/audio/Folder1/2.wav'
audio.export(savePath, format="wav", bitrate="64k")

## **Restart Runtime**

**Silence Padding**

In [ ]:
from pydub import AudioSegment

# Silence before
silence_duration = 500  # Set duration in milliseconds. 1000 is 1 second
save_path = '/content/LIHQ/input/audio/Folder1/1.wav'

silent_segment = AudioSegment.silent(duration=silence_duration)
silent_segment.export(save_path, format="wav")

**Crop your own Speaker Image**

In [ ]:
# Set filepath of your image to be cropped, and where you want the cropped image saved.
%cd '/content/LIHQ/procedures/face_align'
from face_crop import crop_face

# Copying picture I want cropped from drive to colab instance
!gdown --id 1CNN9UA4xCak7d8Z7rQ97PBswFexCxCKF -O '/content/LIHQ/input/face/Donny.jpg'

crop_face(filename = '/content/LIHQ/input/face/Donny.jpg',
          outfile = '/content/LIHQ/input/face/DonnyCrop.jpg')

**Running main script**

In [ ]:
# We want video output very smooth and high quality so we set the frame_int to 3 (100 fps). Takes a long time though.
%cd /content/LIHQ
from runLIHQ import run

run(face='/content/LIHQ/input/face/DonnyCrop.jpg', frame_int=3)

# **Example 4: Create a customized TorToiSe voice, perform postprocessing**

**Generating Customized Audio with TorToiSe**

In [ ]:
# Copy audio files to new TorToiSe folder. 5 clips, around 10 seconds in length each.
%mkdir '/content/LIHQ/tortoise_tts/tortoise/voices/biden'

!gdown --id 1Eco0lWEalc0aG0AYglDUVmiRR9abpEEH -O '/content/LIHQ/tortoise_tts/tortoise/voices/biden/1.wav'
!gdown --id 1cxb5TU484jGCIqAb_7TT7RIxkt97km_m -O '/content/LIHQ/tortoise_tts/tortoise/voices/biden/2.wav'
!gdown --id 1_Mj1ZT9JJ3zHldV2c1d42A4aaEcSRD3q -O '/content/LIHQ/tortoise_tts/tortoise/voices/biden/3.wav'
!gdown --id 1h1cH3qeVWZyBmsquwE439N7s9p_AUScB -O '/content/LIHQ/tortoise_tts/tortoise/voices/biden/4.wav'
!gdown --id 104EcuCcBu5B6DB_-gY-7rWUpaWWzVQyn -O '/content/LIHQ/tortoise_tts/tortoise/voices/biden/5.wav'

In [ ]:
# Run this once. Takes about a minute
%cd /content/LIHQ/tortoise_tts
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

from LIHQ.procedures.tortoise_scripts import tortoise_run, tortoise_combo_run

tts = TextToSpeech()

/content/LIHQ/tortoise_tts


In [ ]:
# Directing voice variable to our new 'biden' voice folder
# When you generate one you like, save it using the cell below
text = "I'm joe biden and I support this message."
voice = 'biden'
preset = "high_quality" #Options: {"ultra_fast", "fast", "standard", "high_quality"}.

gen = tortoise_run(tts, text, voice, preset)
IPython.display.Audio(gen.squeeze(0).cpu(), rate = 24000)

In [ ]:
# Saving audio from above
torchaudio.save('/content/LIHQ/input/audio/Folder1/2.wav', gen.squeeze(0).cpu(), 24000)

In [ ]:
# Silence padding
from pydub import AudioSegment

silence_duration = 300  # Set duration in milliseconds. 1000 is 1 second
save_path = '/content/LIHQ/input/audio/Folder1/1.wav'

silent_segment = AudioSegment.silent(duration=silence_duration)
silent_segment.export(save_path, format="wav")

**Cropping Speaker Image**

In [ ]:
# The output of this cell prints the required parameters to paste the image back onto the same location of the original image.
  # We will need this for the postprocessing
# Set filepath of your image to be cropped, and where you want the cropped image saved.
%cd '/content/LIHQ/procedures/face_align'
from face_crop import crop_face

# Copying picture I want cropped from drive to colab instance
!gdown --id 1oeDMNbQa27stnOGdXgwycXuNyjBj0IRC -O '/content/LIHQ/input/face/biden.jpg'

crop_face(filename = '/content/LIHQ/input/face/biden.jpg',
          outfile = '/content/LIHQ/input/face/bidenCrop.jpg')

**Main Script**

In [ ]:
%cd /content/LIHQ
from runLIHQ import run

run(face='/content/LIHQ/input/face/bidenCrop.jpg', frame_int=2)

**Postprocessing**

**Next three cells are pasting speaker on original full image, resized to fit original location.**

In [ ]:
#Important! If you want to paste the speaker in the same location as the original image, look at printouts of image cropping cell above

speaker_vid = '/content/LIHQ/output/finalVidsOut/Folder1.mp4' # Your speaker video created by LIHQ
background = '/content/LIHQ/input/face/biden.jpg' # Can be image or video (Final video output will be smallest fps between background vid and speaker vid.)
bg_resize = False # Set to False or your desired background size in format: (width, height)
spkr_resize = (221, 221) # Set to False or your desired speaker size in format: (width, height)
offset = [47, 757] # In the form [y, x]. Offset of speaker image onto background, from top left
rotation = 9.37 #Counterclockwise, in degrees
output_path = "/content/biden_static.mp4" # EVERYTHING IN POSTPROCESSING FOLDER GETS DELETED AT THE START OF A NEW RUN!
                                    # So save outside of postprocessing folder.

*Preview*

In [ ]:
# Just running a preview of first frame of final video. Adjust parameters above as needed and rerun
%cd '/content/LIHQ'

from IPython.display import Image
from pathlib import Path

from procedures.matting_scripts import matte_preview
from procedures.matting_scripts import matte_combine

#Delete Video Matting files from previous run (if any)
for path in Path("./output/postprocessing/").glob("**/*"): #Check path with cwd
    if path.is_file():
        path.unlink()

#Generating image data
matte_preview(speaker_vid, background, bg_resize, spkr_resize, offset)

#Creating mask with MODNet
%cd '/content/LIHQ/MODNet'
!python -m demo.image_matting.colab.inference \
        --input-path /content/LIHQ/output/postprocessing/input \
        --output-path /content/LIHQ/output/postprocessing/masks \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt
%cd '..'

#Combining speaker, mask, background
matte_combine(offset, rotation)

Image('./output/postprocessing/output/preview.png')

*Final video creation*

In [ ]:
%cd '/content/LIHQ'

from IPython.display import Image
from pathlib import Path

from procedures.matting_scripts import matte_combine
from procedures.matting_scripts import matte_video

#Delete Video Matting files from previous run (if any)
for path in Path("./output/postprocessing/").glob("**/*"): #Check path with cwd
    if path.is_file():
        path.unlink()

#Generating image data
fps = matte_video(speaker_vid, background, bg_resize, spkr_resize)

#Creating mask with MODNet
%cd '/content/LIHQ/MODNet'
!python -m demo.image_matting.colab.inference \
        --input-path /content/LIHQ/output/postprocessing/input \
        --output-path /content/LIHQ/output/postprocessing/masks \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt
%cd '..'

#Combining speaker, mask, background
matte_combine(offset, rotation)

!ffmpeg -y -framerate {int(fps)} -i '/content/LIHQ/output/postprocessing/output/%5d.png' -i {speaker_vid} -map 0:v:0 -map 1:a:0 -c:v libx264 -pix_fmt yuv420p -b:v 20000k {output_path}

**Next three cells are pasting speaker on generic video background. (Second video)**

In [ ]:
#Downloading our background video
!gdown --id 14dZnMsSLPeFYpbkB0hUZQdp6fCImrNoE -O '/content/LIHQ/input/background/flag.mp4'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=14dZnMsSLPeFYpbkB0hUZQdp6fCImrNoE
To: /content/LIHQ/input/background/flag.mp4
100% 14.0M/14.0M [00:00<00:00, 51.6MB/s]


In [ ]:
speaker_vid = '/content/LIHQ/output/finalVidsOut/Folder1.mp4' # Your speaker video created by LIHQ
background = '/content/LIHQ/input/background/flag.mp4' # Can be image or video (Final video output will be smallest fps between background vid and speaker vid.)
bg_resize = False # Set to False or your desired background size in format: (width, height)
spkr_resize = (1080, 1080) # Set to False or your desired speaker size in format: (width, height)
offset = [0, 420] # In the form [y, x]. Offset of speaker image onto background, from top left
rotation = 0 # Counterclockwise, in degrees
output_path = "/content/biden_video.mp4" # EVERYTHING IN POSTPROCESSING FOLDER GETS DELETED AT THE START OF A NEW RUN!
                                    # So save outside of postprocessing folder.

*Preview*

In [ ]:
# Just running a preview of first frame of final video. Adjust parameters above as needed and rerun
# See output for a suggestion offset of center, center and center, bottom.
%cd '/content/LIHQ'

from IPython.display import Image
from pathlib import Path

from procedures.matting_scripts import matte_preview
from procedures.matting_scripts import matte_combine

#Delete Video Matting files from previous run (if any)
for path in Path("./output/postprocessing/").glob("**/*"): #Check path with cwd
    if path.is_file():
        path.unlink()

#Generating image data
matte_preview(speaker_vid, background, bg_resize, spkr_resize, offset)

#Creating mask with MODNet
%cd '/content/LIHQ/MODNet'
!python -m demo.image_matting.colab.inference \
        --input-path /content/LIHQ/output/postprocessing/input \
        --output-path /content/LIHQ/output/postprocessing/masks \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt
%cd '..'

#Combining speaker, mask, background
matte_combine(offset, rotation)

Image('./output/postprocessing/output/preview.png')

*Final video creation*

In [ ]:
%cd '/content/LIHQ'

from IPython.display import Image
from pathlib import Path

from procedures.matting_scripts import matte_combine
from procedures.matting_scripts import matte_video

#Delete Video Matting files from previous run (if any)
for path in Path("./output/postprocessing/").glob("**/*"): #Check path with cwd
    if path.is_file():
        path.unlink()

#Generating image data
fps = matte_video(speaker_vid, background, bg_resize, spkr_resize)

#Creating mask with MODNet
%cd '/content/LIHQ/MODNet'
!python -m demo.image_matting.colab.inference \
        --input-path /content/LIHQ/output/postprocessing/input \
        --output-path /content/LIHQ/output/postprocessing/masks \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt
%cd '..'

#Combining speaker, mask, background
matte_combine(offset, rotation)

!ffmpeg -y -framerate {int(fps)} -i '/content/LIHQ/output/postprocessing/output/%5d.png' -i {speaker_vid} -map 0:v:0 -map 1:a:0 -c:v libx264 -pix_fmt yuv420p -b:v 20000k {output_path}